In [11]:
from cv2 import imread, resize, imshow, destroyAllWindows, waitKey
import numpy as np
import torch
import glob, os
import json
import xml.etree.cElementTree as ET
from parameters import IMAGE_HEIGHT, IMAGE_WIDTH, device


class CustomImageDataset:
    def __init__(
        self, base_path="Single_Labels", img_dir=glob.glob("Line_data/Images/*/*/*"),
    ):

        self.base_path = base_path
        print("Label lenght",(self.base_path))
        self.img_dir = img_dir
        print("No of Image",len(self.img_dir))

    def Load_Image_Label(self, image_path):
        # Open the image file
        label = tuple()
        json_path = os.path.join(
            self.base_path, image_path.split("\\")[-1][:-4] + ".json"
        )
        with open(json_path, "r") as json_file:
            label = json.load(json_file)
        img = imread(image_path, 0)
        img = 255 - img
        img_height, img_width = img.shape[0], img.shape[1]
        n_repeats = int(np.ceil(IMAGE_WIDTH / img_width))
        padded_image = np.concatenate([img] * n_repeats, axis=1)
        padded_image = padded_image[:IMAGE_HEIGHT, :IMAGE_WIDTH]
        resized_img = resize(padded_image, (IMAGE_WIDTH, IMAGE_HEIGHT))
        return (resized_img, label)
        # plt.imshow(img)
        # plt.show()

    def __len__(self):
        return len(self.img_dir)

    def __getitem__(self, idx):
        print(idx)
        # import pdb;pdb.set_trace()
        Image, Labels = self.Load_Image_Label(self.img_dir[idx])
        return torch.tensor(Image, device=device).float(), Labels
        # return Image,Labels


In [12]:
TextDatasetObj = CustomImageDataset()
no_workers=0
dataset = torch.utils.data.DataLoader(
TextDatasetObj, batch_size=15, num_workers=no_workers
)
if len(dataset) == 0:
    print("The train_loader is empty.")
    StopIteration

Label lenght Single_Labels
No of Image 0
The train_loader is empty.


In [6]:
import tqdm
for Image, Label in tqdm.tqdm(dataset):
    label = pad_str(Label[0])
    print(f"{len(label)=} {len(label[0][0])=}")

0it [00:00, ?it/s]


In [1]:
from models import ImageEncoder

FileNotFoundError: [Errno 2] No such file or directory: 'corpora_english/brown-azAZ.tr'

In [2]:
import torch
import torch.nn as nn

def make_layers_custom(cfg, batch_norm=False):
    layers = []
    in_channels = 10
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.InstanceNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

cfg_custom = {
    'custom': [64, 64, 'M', 128, 128, 'M', 256, 'M', 512, 512, 'M', 512, 512],
}

class CustomVGG(nn.Module):
    def __init__(self, features, init_weights=True):
        super(CustomVGG, self).__init__()
        self.features = features
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

# Create the custom VGG model
custom_vgg_model = CustomVGG(make_layers_custom(cfg_custom['custom'], batch_norm=True))

# Example usage with an input tensor of size [4, 10, 2, 512]
input_tensor = torch.randn(4, 10, 2, 512)
output = custom_vgg_model(input_tensor)
print(output.size())  # Output tensor size


RuntimeError: Given input size: (128x1x256). Calculated output size: (128x0x128). Output size is too small

In [8]:
import tqdm
l=torch.arange(10)

In [9]:

for i,index in tqdm.tqdm(enumerate(l)):
    print(i,index)

10it [00:00, 394.76it/s]

0 tensor(0)
1 tensor(1)
2 tensor(2)
3 tensor(3)
4 tensor(4)
5 tensor(5)
6 tensor(6)
7 tensor(7)
8 tensor(8)
9 tensor(9)
